In [1]:
#https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers, callbacks

Using TensorFlow backend.


In [2]:
#1. Dataset preparation3. Model Building
# load the dataset
data = open('data/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [3]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'],test_size=0.1)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


In [4]:
print(train_y)

[0 0 0 ... 1 1 1]


In [5]:
#2.1 Count Vectors as features
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)


In [6]:
xtrain_count.shape

(9000, 5000)

In [7]:
#2.2 TF-IDF Vectors as features
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [8]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,5), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,5), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

In [9]:
#2.3 Word Embeddings
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [10]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.0897      0.016      -0.0571     ...  0.1559     -0.0254
  -0.0259    ]
 [-0.0314      0.0149     -0.0205     ...  0.098       0.0893
   0.0148    ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.0897     -0.0682      0.0726     ...  0.206       0.0839
   0.1557    ]
 [-0.0857      0.0475      0.14489999 ...  0.0586      0.11
   0.0743    ]]


In [ ]:
#2.4 Text / NLP based features
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [ ]:
#2.5 Topic Models as features
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))


In [9]:
#3. Model Building
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)    
    #print(predictions[0:20])
    #print(valid_y[0:20])
    return metrics.accuracy_score(predictions, valid_y)

In [10]:
#3.1 Naive Bayes
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.835
NB, WordLevel TF-IDF:  0.85
NB, N-Gram Vectors:  0.829
NB, CharLevel Vectors:  0.815


NB, Count Vectors:  0.8488
NB, WordLevel TF-IDF:  0.8524
NB, N-Gram Vectors:  0.8448
NB, CharLevel Vectors:  0.8108

In [11]:
#3.2 Linear Classifier
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.868
LR, WordLevel TF-IDF:  0.873
LR, N-Gram Vectors:  0.835
LR, CharLevel Vectors:  0.839


In [12]:
accuracy = train_model(linear_model.LogisticRegression(penalty='l2',C=5.0), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

LR, WordLevel TF-IDF:  0.874


With test size of 0.1 nest LR model is giving accuracy of 88.2 , we can also see how it works with word count dataset

In [13]:
accuracy = train_model(linear_model.LogisticRegression(C=3.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

LR, WordLevel TF-IDF:  0.873


In [14]:
accuracy = train_model(linear_model.LogisticRegression(), train_seq_x, train_y, valid_seq_x)
print ("LR, Embedding: ", accuracy)

NameError: name 'train_seq_x' is not defined

LR, Count Vectors:  0.8688
LR, WordLevel TF-IDF:  0.872
LR, N-Gram Vectors:  0.8364
LR, CharLevel Vectors:  0.847
best LR, WordLevel TF-IDF:  0.882
LR, Embedding:  0.4888

In [15]:
accuracy = train_model(svm.SVC(kernel='linear'), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)
accuracy = train_model(svm.SVC(kernel='linear'), xtrain_count, train_y, xvalid_count)
print ("SVM,WordLevel WordCount: ", accuracy)
accuracy = train_model(svm.SVC(kernel='linear'), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM,WordLevel TF-IDF: ", accuracy)
accuracy = train_model(svm.SVC(kernel='linear'), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM,CharLevel Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.836
SVM,WordLevel WordCount:  0.833
SVM,WordLevel TF-IDF:  0.876
SVM,CharLevel Vectors:  0.854


SVM, N-Gram Vectors:  0.8168
SVM,WordLevel WordCount:  0.8464
SVM,WordLevel TF-IDF:  0.87
SVM,CharLevel Vectors:  0.8352

with test size of 0.1
SVM, N-Gram Vectors:  0.836
SVM,WordLevel WordCount:  0.833
SVM,WordLevel TF-IDF:  0.876
SVM,CharLevel Vectors:  0.854

In [21]:
#3.3 Implementing a SVM Model
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(C=100.0, gamma= 0.001), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)
accuracy = train_model(svm.SVC(C=100.0, gamma= 0.001), xtrain_count, train_y, xvalid_count)
print ("SVM,Count Vectors: ", accuracy)
accuracy = train_model(svm.SVC(C=100.0, gamma= 0.001), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM,WordLevel TF-IDF: ", accuracy)
accuracy = train_model(svm.SVC(C=100.0, gamma= 0.001), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM,CharLevel Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.8248
SVM,Count Vectors:  0.8488
SVM,WordLevel TF-IDF:  0.8568
SVM,CharLevel Vectors:  0.8312


SVM, N-Gram Vectors:  0.84
SVM,Count Vectors:  0.856
SVM,WordLevel TF-IDF:  0.8704
SVM,CharLevel Vectors:  0.836

In [ ]:
#3.4 Bagging Model
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100,criterion='entropy'), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)
# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100,criterion='entropy'), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

RF, Count Vectors:  0.8464
RF, WordLevel TF-IDF:  0.8344

In [101]:
#3.5 Boosting Model
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(max_depth=7,
                           min_child_weight=1,
                           learning_rate=0.1,
                           n_estimators=500,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=1,
                           missing=None), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier( learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 0]
[0 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 1 0]
Xgb, Count Vectors:  0.8452


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0]
[0 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 1 0]
Xgb, WordLevel TF-IDF:  0.8288
[0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 1 0]
[0 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 1 0]
Xgb, CharLevel Vectors:  0.8048


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=1,
 colsample_bytree=1,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, TF IDF Vectors: ", accuracy)

Xgb, TF IDF Vectors:  0.857


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [47]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, Naive TF IDF Vectors: ", accuracy)

Xgb, Naive TF IDF Vectors:  0.815


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, Count Vectors: 0.8584
Xgb, WordLevel TF-IDF: 0.8356
Xgb, CharLevel Vectors:  0.8152
Xgb, TF IDF Vectors:  0.8284
Xgb, Naive TF IDF Vectors:  0.7916

with test size of 0.1
Xgb, TF IDF Vectors:  0.857
Xgb, Naive TF IDF Vectors:  0.815

In [16]:
from sklearn.model_selection import GridSearchCV
import numpy as np


In [17]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge

In [18]:
linear_model.LogisticRegression().get_params().keys()

dict_keys(['n_jobs', 'multi_class', 'fit_intercept', 'warm_start', 'C', 'solver', 'random_state', 'verbose', 'max_iter', 'class_weight', 'tol', 'penalty', 'dual', 'intercept_scaling'])

In [19]:
gs_clf = GridSearchCV(linear_model.LogisticRegression(), grid,cv=10)
gs_clf = gs_clf.fit(xtrain_tfidf, train_y)

In [20]:
gs_clf.best_score_

0.8698888888888889

In [23]:
gs_clf.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.51000, std: 0.00000, params: {'penalty': 'l1', 'C': 0.001},
 mean: 0.51233, std: 0.00225, params: {'penalty': 'l2', 'C': 0.001},
 mean: 0.51000, std: 0.00000, params: {'penalty': 'l1', 'C': 0.01},
 mean: 0.77911, std: 0.00906, params: {'penalty': 'l2', 'C': 0.01},
 mean: 0.74689, std: 0.01393, params: {'penalty': 'l1', 'C': 0.1},
 mean: 0.83522, std: 0.01089, params: {'penalty': 'l2', 'C': 0.1},
 mean: 0.85589, std: 0.01231, params: {'penalty': 'l1', 'C': 1.0},
 mean: 0.86989, std: 0.01019, params: {'penalty': 'l2', 'C': 1.0},
 mean: 0.84944, std: 0.01480, params: {'penalty': 'l1', 'C': 10.0},
 mean: 0.86844, std: 0.01044, params: {'penalty': 'l2', 'C': 10.0},
 mean: 0.83344, std: 0.01140, params: {'penalty': 'l1', 'C': 100.0},
 mean: 0.84922, std: 0.01082, params: {'penalty': 'l2', 'C': 100.0},
 mean: 0.83078, std: 0.01159, params: {'penalty': 'l1', 'C': 1000.0},
 mean: 0.84000, std: 0.01230, params: {'penalty': 'l2', 'C': 1000.0}]

In [21]:
gs_clf.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [22]:
gs_clf.best_estimator_gs_clf.

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [117]:
param_test = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}

In [125]:
gsearch = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=1, colsample_bytree=1,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)

In [126]:
train_model4 = gsearch.fit(xtrain_tfidf, train_y)

In [127]:
train_model4.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.90086, std: 0.00353, params: {'max_depth': 4, 'min_child_weight': 4},
 mean: 0.90122, std: 0.00439, params: {'max_depth': 4, 'min_child_weight': 5},
 mean: 0.90054, std: 0.00325, params: {'max_depth': 4, 'min_child_weight': 6},
 mean: 0.90449, std: 0.00437, params: {'max_depth': 5, 'min_child_weight': 4},
 mean: 0.90474, std: 0.00558, params: {'max_depth': 5, 'min_child_weight': 5},
 mean: 0.90437, std: 0.00324, params: {'max_depth': 5, 'min_child_weight': 6},
 mean: 0.90732, std: 0.00553, params: {'max_depth': 6, 'min_child_weight': 4},
 mean: 0.90706, std: 0.00379, params: {'max_depth': 6, 'min_child_weight': 5},
 mean: 0.90568, std: 0.00373, params: {'max_depth': 6, 'min_child_weight': 6}]

In [128]:
train_model4.best_score_

0.9073239176666685

In [132]:
train_model4.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=4, missing=None, n_estimators=140,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=1)

In [46]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=4, missing=None, n_estimators=140,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=1), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb,Grid search TF IDF Vectors: ", accuracy)

Xgb,Grid search TF IDF Vectors:  0.84


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb,Grid search TF IDF Vectors:  0.8156

with test size of 0.1 
Xgb,Grid search TF IDF Vectors:  0.84

In [26]:
train_y.shape

(9000,)

In [11]:
#3. Model Building
def train_model_new(classifier, feature_vector_train, label, feature_vector_valid, epoch, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label,epochs=epoch,validation_data=(feature_vector_valid, valid_y))
    
    print(classifier.evaluate(feature_vector_valid,valid_y))
    # predict the labels on validation dataset
    #predictions = classifier.predict(feature_vector_valid,verbose=1) 
    #print(predictions[0:20])
    #predictions = [1 if x >= 0.05 else 0 for x in predictions]
    #if is_neural_net:
        #predictions = predictions.argmax(axis=1)
    
    #print(predictions[0:20])
    #print(valid_y[0:20])
    #print(metrics.accuracy_score(valid_y,predictions))
    #return metrics.roc_auc_score(valid_y,predictions)
    return classifier.evaluate(feature_vector_valid,valid_y)

In [12]:
#3.6 Shallow Neural Networks
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(1000, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model_new(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram,10, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 36s 4ms/step - loss: 0.4370 - acc: 0.7998 - val_loss: 0.3455 - val_acc: 0.8470
Epoch 2/10
9000/9000 [==============================] - 34s 4ms/step - loss: 0.2318 - acc: 0.9033 - val_loss: 0.3888 - val_acc: 0.8390
Epoch 3/10
9000/9000 [==============================] - 35s 4ms/step - loss: 0.1377 - acc: 0.9488 - val_loss: 0.4970 - val_acc: 0.8270
Epoch 4/10
9000/9000 [==============================] - 34s 4ms/step - loss: 0.0687 - acc: 0.9793 - val_loss: 0.6039 - val_acc: 0.8240
Epoch 5/10
9000/9000 [==============================] - 33s 4ms/step - loss: 0.0293 - acc: 0.9936 - val_loss: 0.7255 - val_acc: 0.8250
Epoch 6/10
9000/9000 [==============================] - 33s 4ms/step - loss: 0.0121 - acc: 0.9986 - val_loss: 0.8208 - val_acc: 0.8230
Epoch 7/10
9000/9000 [==============================] - 34s 4ms/step - loss: 0.0065 - acc: 0.9988 - val_loss: 0.8898 - val_acc: 0.8240
Epoch 8

In [ ]:
Epoch 1/4
9000/9000 [==============================] - 35s 4ms/step - loss: 0.4346 - acc: 0.8030
Epoch 2/4
9000/9000 [==============================] - 33s 4ms/step - loss: 0.2230 - acc: 0.9084
Epoch 3/4
9000/9000 [==============================] - 33s 4ms/step - loss: 0.1362 - acc: 0.9504
Epoch 4/4
9000/9000 [==============================] - 37s 4ms/step - loss: 0.0689 - acc: 0.9773
NN, Ngram Level TF IDF Vectors 0.8202636779425264

In [13]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              5001000   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001      
Total params: 5,002,001
Trainable params: 5,002,001
Non-trainable params: 0
_________________________________________________________________


In [14]:
#3.7 Deep Neural Networks
#3.7.1 Convolutional Neural Network
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.5)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(300, 3, activation="relu")(embedding_layer)

    #pooling_layer = layers.AveragePooling1D()(conv_layer)
    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    

    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.5)(output_layer1)
    output_layer11 = layers.Dense(25, activation="relu")(output_layer1)
    output_layer11 = layers.Dropout(0.5)(output_layer11)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer11)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    
    return model

classifier = create_cnn()
accuracy = train_model_new(classifier, train_seq_x, train_y, valid_seq_x, 30, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 36s 4ms/step - loss: 0.6962 - acc: 0.5183 - val_loss: 0.6716 - val_acc: 0.5200
Epoch 2/30
9000/9000 [==============================] - 31s 3ms/step - loss: 0.5693 - acc: 0.7046 - val_loss: 0.3803 - val_acc: 0.8370
Epoch 3/30
9000/9000 [==============================] - 31s 3ms/step - loss: 0.4477 - acc: 0.8032 - val_loss: 0.3386 - val_acc: 0.8500
Epoch 4/30
9000/9000 [==============================] - 31s 3ms/step - loss: 0.4122 - acc: 0.8199 - val_loss: 0.3243 - val_acc: 0.8670
Epoch 5/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.3826 - acc: 0.8362 - val_loss: 0.3358 - val_acc: 0.8660
Epoch 6/30
9000/9000 [==============================] - 29s 3ms/step - loss: 0.3559 - acc: 0.8493 - val_loss: 0.3262 - val_acc: 0.8530
Epoch 7/30
9000/9000 [==============================] - 33s 4ms/step - loss: 0.3375 - acc: 0.8634 - val_loss: 0.3176 - val_acc: 0.8710
Epoch 8

In [ ]:
Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 17s 2ms/step - loss: 0.5483 - acc: 0.7082 - val_loss: 0.3757 - val_acc: 0.8440
Epoch 2/10
9000/9000 [==============================] - 17s 2ms/step - loss: 0.3739 - acc: 0.8374 - val_loss: 0.3283 - val_acc: 0.8660
Epoch 3/10
9000/9000 [==============================] - 16s 2ms/step - loss: 0.3168 - acc: 0.8664 - val_loss: 0.3108 - val_acc: 0.8660
Epoch 4/10
9000/9000 [==============================] - 17s 2ms/step - loss: 0.2672 - acc: 0.8901 - val_loss: 0.3203 - val_acc: 0.8580
Epoch 5/10
9000/9000 [==============================] - 17s 2ms/step - loss: 0.2414 - acc: 0.8999 - val_loss: 0.3112 - val_acc: 0.8650
Epoch 6/10
9000/9000 [==============================] - 17s 2ms/step - loss: 0.2031 - acc: 0.9238 - val_loss: 0.3186 - val_acc: 0.8680
Epoch 7/10
9000/9000 [==============================] - 18s 2ms/step - loss: 0.1634 - acc: 0.9372 - val_loss: 0.3192 - val_acc: 0.8680
Epoch 8/10
9000/9000 [==============================] - 18s 2ms/step - loss: 0.1475 - acc: 0.9441 - val_loss: 0.3090 - val_acc: 0.8640
Epoch 9/10
9000/9000 [==============================] - 19s 2ms/step - loss: 0.1086 - acc: 0.9607 - val_loss: 0.3466 - val_acc: 0.8700
Epoch 10/10
9000/9000 [==============================] - 18s 2ms/step - loss: 0.0918 - acc: 0.9664 - val_loss: 0.3961 - val_acc: 0.8570
1000/1000 [==============================] - 1s 645us/step
[0.396127006329596, 0.857]
1000/1000 [==============================] - 1s 697us/step
CNN, Word Embeddings [0.396127006329596, 0.857]

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 24s 3ms/step - loss: 0.5527 - acc: 0.7001 - val_loss: 0.3671 - val_acc: 0.8490
Epoch 2/10
9000/9000 [==============================] - 24s 3ms/step - loss: 0.3656 - acc: 0.8398 - val_loss: 0.3298 - val_acc: 0.8690
Epoch 3/10
9000/9000 [==============================] - 23s 3ms/step - loss: 0.2964 - acc: 0.8787 - val_loss: 0.3103 - val_acc: 0.8670
Epoch 4/10
9000/9000 [==============================] - 23s 3ms/step - loss: 0.2445 - acc: 0.8991 - val_loss: 0.3340 - val_acc: 0.8660
Epoch 5/10
9000/9000 [==============================] - 23s 3ms/step - loss: 0.1912 - acc: 0.9258 - val_loss: 0.3391 - val_acc: 0.8710
Epoch 6/10
9000/9000 [==============================] - 24s 3ms/step - loss: 0.1538 - acc: 0.9416 - val_loss: 0.3388 - val_acc: 0.8750
Epoch 7/10
9000/9000 [==============================] - 24s 3ms/step - loss: 0.1161 - acc: 0.9546 - val_loss: 0.3609 - val_acc: 0.8700
Epoch 8/10
9000/9000 [==============================] - 25s 3ms/step - loss: 0.0933 - acc: 0.9661 - val_loss: 0.3731 - val_acc: 0.8730
Epoch 9/10
9000/9000 [==============================] - 25s 3ms/step - loss: 0.0842 - acc: 0.9690 - val_loss: 0.3969 - val_acc: 0.8810
Epoch 10/10
9000/9000 [==============================] - 25s 3ms/step - loss: 0.0619 - acc: 0.9777 - val_loss: 0.4201 - val_acc: 0.8830
1000/1000 [==============================] - 1s 929us/step
[0.4201457166671753, 0.883]
1000/1000 [==============================] - 1s 967us/step
CNN, Word Embeddings [0.4201457166671753, 0.883]

Layer (type)                 Output Shape              Param #   
=================================================================
input_6 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 70, 300)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 68, 200)           180200    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 11        
=================================================================
Total params: 10,242,321
Trainable params: 201,321
Non-trainable params: 10,041,000
_________________________________________________________________

In [15]:
Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 36s 4ms/step - loss: 0.6962 - acc: 0.5183 - val_loss: 0.6716 - val_acc: 0.5200
Epoch 2/30
9000/9000 [==============================] - 31s 3ms/step - loss: 0.5693 - acc: 0.7046 - val_loss: 0.3803 - val_acc: 0.8370
Epoch 3/30
9000/9000 [==============================] - 31s 3ms/step - loss: 0.4477 - acc: 0.8032 - val_loss: 0.3386 - val_acc: 0.8500
Epoch 4/30
9000/9000 [==============================] - 31s 3ms/step - loss: 0.4122 - acc: 0.8199 - val_loss: 0.3243 - val_acc: 0.8670
Epoch 5/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.3826 - acc: 0.8362 - val_loss: 0.3358 - val_acc: 0.8660
Epoch 6/30
9000/9000 [==============================] - 29s 3ms/step - loss: 0.3559 - acc: 0.8493 - val_loss: 0.3262 - val_acc: 0.8530
Epoch 7/30
9000/9000 [==============================] - 33s 4ms/step - loss: 0.3375 - acc: 0.8634 - val_loss: 0.3176 - val_acc: 0.8710
Epoch 8/30
9000/9000 [==============================] - 35s 4ms/step - loss: 0.3165 - acc: 0.8746 - val_loss: 0.3117 - val_acc: 0.8770
Epoch 9/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.2984 - acc: 0.8828 - val_loss: 0.3015 - val_acc: 0.8690
Epoch 10/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.2838 - acc: 0.8870 - val_loss: 0.3183 - val_acc: 0.8740
Epoch 11/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.2681 - acc: 0.8920 - val_loss: 0.3006 - val_acc: 0.8740
Epoch 12/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.2471 - acc: 0.9063 - val_loss: 0.3934 - val_acc: 0.8450
Epoch 13/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.2264 - acc: 0.9123 - val_loss: 0.3241 - val_acc: 0.8790
Epoch 14/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.2107 - acc: 0.9200 - val_loss: 0.3179 - val_acc: 0.8750
Epoch 15/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.2134 - acc: 0.9174 - val_loss: 0.3501 - val_acc: 0.8730
Epoch 16/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1978 - acc: 0.9247 - val_loss: 0.3861 - val_acc: 0.8680
Epoch 17/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1658 - acc: 0.9403 - val_loss: 0.3758 - val_acc: 0.8790
Epoch 18/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1756 - acc: 0.9330 - val_loss: 0.3616 - val_acc: 0.8800
Epoch 19/30
9000/9000 [==============================] - 34s 4ms/step - loss: 0.1642 - acc: 0.9363 - val_loss: 0.3998 - val_acc: 0.8760
Epoch 20/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1587 - acc: 0.9421 - val_loss: 0.3945 - val_acc: 0.8730
Epoch 21/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1510 - acc: 0.9443 - val_loss: 0.4184 - val_acc: 0.8710
Epoch 22/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1443 - acc: 0.9486 - val_loss: 0.4138 - val_acc: 0.8700
Epoch 23/30
9000/9000 [==============================] - 33s 4ms/step - loss: 0.1455 - acc: 0.9470 - val_loss: 0.4113 - val_acc: 0.8740
Epoch 24/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1294 - acc: 0.9520 - val_loss: 0.4233 - val_acc: 0.8710
Epoch 25/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1321 - acc: 0.9491 - val_loss: 0.4638 - val_acc: 0.8710
Epoch 26/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1274 - acc: 0.9556 - val_loss: 0.4327 - val_acc: 0.8820
Epoch 27/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1308 - acc: 0.9518 - val_loss: 0.4471 - val_acc: 0.8800
Epoch 28/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1253 - acc: 0.9572 - val_loss: 0.4211 - val_acc: 0.8890
Epoch 29/30
9000/9000 [==============================] - 32s 4ms/step - loss: 0.1190 - acc: 0.9579 - val_loss: 0.4262 - val_acc: 0.8790
Epoch 30/30
9000/9000 [==============================] - 33s 4ms/step - loss: 0.1148 - acc: 0.9600 - val_loss: 0.4280 - val_acc: 0.8750
1000/1000 [==============================] - 1s 1ms/step
[0.42802009439468386, 0.875]
1000/1000 [==============================] - 1s 1ms/step
CNN, Word Embeddings [0.42802009439468386, 0.875]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_4 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 70, 300)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 68, 300)           270300    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               30100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 25)                2525      
_________________________________________________________________
dropout_6 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 26        
=================================================================
Total params: 10,343,951
Trainable params: 302,951
Non-trainable params: 10,041,000
_________________________________________________________________

SyntaxError: invalid syntax (<ipython-input-15-8a0a6a515563>, line 1)

In [16]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 70, 300)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 68, 300)           270300    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               30100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
__________

In [14]:
#3.7.2 Recurrent Neural Network – LSTM
# Add an Input Layer
def create_rnn_lstm():
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(300)(embedding_layer)
    lstm_layer = layers.Dropout(0.3)(lstm_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.3)(output_layer1)
    output_layer11 = layers.Dense(10, activation="relu")(output_layer1)
    output_layer11 = layers.Dropout(0.3)(output_layer11)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer11)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model_new(classifier, train_seq_x, train_y, valid_seq_x, 20, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracy)

Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 117s 13ms/step - loss: 0.6170 - acc: 0.6604 - val_loss: 0.4988 - val_acc: 0.7720
Epoch 2/20
9000/9000 [==============================] - 115s 13ms/step - loss: 0.5343 - acc: 0.7530 - val_loss: 0.4759 - val_acc: 0.8030
Epoch 3/20
9000/9000 [==============================] - 115s 13ms/step - loss: 0.4945 - acc: 0.7779 - val_loss: 0.4256 - val_acc: 0.8010
Epoch 4/20
9000/9000 [==============================] - 117s 13ms/step - loss: 0.4662 - acc: 0.7936 - val_loss: 0.4449 - val_acc: 0.8230
Epoch 5/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.4283 - acc: 0.8131 - val_loss: 0.3773 - val_acc: 0.8410
Epoch 6/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.4078 - acc: 0.8219 - val_loss: 0.3663 - val_acc: 0.8490
Epoch 7/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.3868 - acc: 0.8356 - val_loss: 0.3500 - val_acc: 

In [ ]:
Train on 9000 samples, validate on 1000 samples
Epoch 1/5
9000/9000 [==============================] - 38s 4ms/step - loss: 0.6367 - acc: 0.6231 - val_loss: 0.5328 - val_acc: 0.7560
Epoch 2/5
9000/9000 [==============================] - 34s 4ms/step - loss: 0.5233 - acc: 0.7596 - val_loss: 0.4308 - val_acc: 0.8130
Epoch 3/5
9000/9000 [==============================] - 34s 4ms/step - loss: 0.4925 - acc: 0.7810 - val_loss: 0.4216 - val_acc: 0.8220
Epoch 4/5
9000/9000 [==============================] - 34s 4ms/step - loss: 0.4643 - acc: 0.7889 - val_loss: 0.4107 - val_acc: 0.8240
Epoch 5/5
9000/9000 [==============================] - 34s 4ms/step - loss: 0.4509 - acc: 0.7969 - val_loss: 0.3704 - val_acc: 0.8270
1000/1000 [==============================] - 2s 2ms/step
[0.37039316987991333, 0.827]
1000/1000 [==============================] - 2s 2ms/step
RNN-LSTM, Word Embeddings [0.37039316987991333, 0.827]

Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 117s 13ms/step - loss: 0.6170 - acc: 0.6604 - val_loss: 0.4988 - val_acc: 0.7720
Epoch 2/20
9000/9000 [==============================] - 115s 13ms/step - loss: 0.5343 - acc: 0.7530 - val_loss: 0.4759 - val_acc: 0.8030
Epoch 3/20
9000/9000 [==============================] - 115s 13ms/step - loss: 0.4945 - acc: 0.7779 - val_loss: 0.4256 - val_acc: 0.8010
Epoch 4/20
9000/9000 [==============================] - 117s 13ms/step - loss: 0.4662 - acc: 0.7936 - val_loss: 0.4449 - val_acc: 0.8230
Epoch 5/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.4283 - acc: 0.8131 - val_loss: 0.3773 - val_acc: 0.8410
Epoch 6/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.4078 - acc: 0.8219 - val_loss: 0.3663 - val_acc: 0.8490
Epoch 7/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.3868 - acc: 0.8356 - val_loss: 0.3500 - val_acc: 0.8660
Epoch 8/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.3583 - acc: 0.8498 - val_loss: 0.3367 - val_acc: 0.8480
Epoch 9/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.3550 - acc: 0.8504 - val_loss: 0.3321 - val_acc: 0.8620
Epoch 10/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.3288 - acc: 0.8641 - val_loss: 0.3392 - val_acc: 0.8580
Epoch 11/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.3177 - acc: 0.8710 - val_loss: 0.3239 - val_acc: 0.8570
Epoch 12/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.2962 - acc: 0.8773 - val_loss: 0.3282 - val_acc: 0.8720
Epoch 13/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.2949 - acc: 0.8793 - val_loss: 0.3273 - val_acc: 0.8630
Epoch 14/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.2769 - acc: 0.8890 - val_loss: 0.3853 - val_acc: 0.8580
Epoch 15/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.2615 - acc: 0.8953 - val_loss: 0.3556 - val_acc: 0.8680
Epoch 16/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.2451 - acc: 0.9016 - val_loss: 0.3565 - val_acc: 0.8610
Epoch 17/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.2294 - acc: 0.9107 - val_loss: 0.3595 - val_acc: 0.8550
Epoch 18/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.2193 - acc: 0.9114 - val_loss: 0.4043 - val_acc: 0.8670
Epoch 19/20
9000/9000 [==============================] - 114s 13ms/step - loss: 0.2048 - acc: 0.9218 - val_loss: 0.4984 - val_acc: 0.8570
Epoch 20/20
9000/9000 [==============================] - 113s 13ms/step - loss: 0.1935 - acc: 0.9241 - val_loss: 0.4303 - val_acc: 0.8650
1000/1000 [==============================] - 4s 4ms/step
[0.43025758779048917, 0.865]
1000/1000 [==============================] - 5s 5ms/step
RNN-LSTM, Word Embeddings [0.43025758779048917, 0.865]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_2 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 70, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
=================================================================
Total params: 10,793,321
Trainable params: 752,321
Non-trainable params: 10,041,000
_________________________________________________________________

In [15]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 70, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [35]:
#3.7.3 Recurrent Neural Network – GRU
def create_rnn_gru():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.4)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(300)(embedding_layer)
    lstm_layer = layers.Dropout(0.4)(lstm_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.4)(output_layer1)
    output_layer11 = layers.Dense(10, activation="relu")(output_layer1)
    output_layer11 = layers.Dropout(0.4)(output_layer11)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer11)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',metrics=['acc'])
    
    return model

classifier = create_rnn_gru()
accuracy = train_model_new(classifier, train_seq_x, train_y, valid_seq_x, 30,is_neural_net=True)
print ("RNN-GRU, Word Embeddings",  accuracy)

Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 93s 10ms/step - loss: 0.6576 - acc: 0.6053 - val_loss: 0.5076 - val_acc: 0.7520
Epoch 2/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.4966 - acc: 0.7743 - val_loss: 0.3860 - val_acc: 0.8390
Epoch 3/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.4230 - acc: 0.8187 - val_loss: 0.3962 - val_acc: 0.8030
Epoch 4/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.4052 - acc: 0.8273 - val_loss: 0.3307 - val_acc: 0.8590
Epoch 5/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3826 - acc: 0.8329 - val_loss: 0.3193 - val_acc: 0.8660
Epoch 6/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3723 - acc: 0.8399 - val_loss: 0.3342 - val_acc: 0.8700
Epoch 7/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3559 - acc: 0.8503 - val_loss: 0.3354 - val_acc: 0.8530


In [ ]:
Train on 9000 samples, validate on 1000 samples
Epoch 1/5
9000/9000 [==============================] - 33s 4ms/step - loss: 0.6062 - acc: 0.6626 - val_loss: 0.5270 - val_acc: 0.7250
Epoch 2/5
9000/9000 [==============================] - 32s 4ms/step - loss: 0.4541 - acc: 0.7907 - val_loss: 0.3607 - val_acc: 0.8470
Epoch 3/5
9000/9000 [==============================] - 32s 4ms/step - loss: 0.3892 - acc: 0.8287 - val_loss: 0.3357 - val_acc: 0.8550
Epoch 4/5
9000/9000 [==============================] - 32s 4ms/step - loss: 0.3634 - acc: 0.8430 - val_loss: 0.3167 - val_acc: 0.8550
Epoch 5/5
9000/9000 [==============================] - 31s 3ms/step - loss: 0.3453 - acc: 0.8480 - val_loss: 0.3117 - val_acc: 0.8630
1000/1000 [==============================] - 1s 1ms/step
[0.3116701068878174, 0.863]
1000/1000 [==============================] - 2s 2ms/step
RNN-GRU, Word Embeddings [0.3116701068878174, 0.863]


Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 103s 11ms/step - loss: 0.6402 - acc: 0.6390 - val_loss: 0.5746 - val_acc: 0.6940
Epoch 2/10
9000/9000 [==============================] - 97s 11ms/step - loss: 0.4652 - acc: 0.7909 - val_loss: 0.3335 - val_acc: 0.8560
Epoch 3/10
9000/9000 [==============================] - 99s 11ms/step - loss: 0.4017 - acc: 0.8303 - val_loss: 0.3395 - val_acc: 0.8530
Epoch 4/10
9000/9000 [==============================] - 94s 10ms/step - loss: 0.3710 - acc: 0.8412 - val_loss: 0.3257 - val_acc: 0.8610
Epoch 5/10
9000/9000 [==============================] - 92s 10ms/step - loss: 0.3634 - acc: 0.8513 - val_loss: 0.3402 - val_acc: 0.8500
Epoch 6/10
9000/9000 [==============================] - 95s 11ms/step - loss: 0.3361 - acc: 0.8610 - val_loss: 0.3340 - val_acc: 0.8500
Epoch 7/10
9000/9000 [==============================] - 96s 11ms/step - loss: 0.3266 - acc: 0.8630 - val_loss: 0.3249 - val_acc: 0.8520
Epoch 8/10
9000/9000 [==============================] - 102s 11ms/step - loss: 0.3007 - acc: 0.8737 - val_loss: 0.3025 - val_acc: 0.8730
Epoch 9/10
9000/9000 [==============================] - 102s 11ms/step - loss: 0.2868 - acc: 0.8821 - val_loss: 0.3273 - val_acc: 0.8610
Epoch 10/10
9000/9000 [==============================] - 102s 11ms/step - loss: 0.2631 - acc: 0.8944 - val_loss: 0.3416 - val_acc: 0.8400
1000/1000 [==============================] - 4s 4ms/step
[0.34156269550323487, 0.84]
1000/1000 [==============================] - 4s 4ms/step
RNN-GRU, Word Embeddings [0.34156269550323487, 0.84]

Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 93s 10ms/step - loss: 0.6576 - acc: 0.6053 - val_loss: 0.5076 - val_acc: 0.7520
Epoch 2/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.4966 - acc: 0.7743 - val_loss: 0.3860 - val_acc: 0.8390
Epoch 3/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.4230 - acc: 0.8187 - val_loss: 0.3962 - val_acc: 0.8030
Epoch 4/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.4052 - acc: 0.8273 - val_loss: 0.3307 - val_acc: 0.8590
Epoch 5/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3826 - acc: 0.8329 - val_loss: 0.3193 - val_acc: 0.8660
Epoch 6/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3723 - acc: 0.8399 - val_loss: 0.3342 - val_acc: 0.8700
Epoch 7/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3559 - acc: 0.8503 - val_loss: 0.3354 - val_acc: 0.8530
Epoch 8/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3387 - acc: 0.8571 - val_loss: 0.3361 - val_acc: 0.8530
Epoch 9/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3306 - acc: 0.8640 - val_loss: 0.3082 - val_acc: 0.8610
Epoch 10/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3120 - acc: 0.8711 - val_loss: 0.3129 - val_acc: 0.8590
Epoch 11/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.3002 - acc: 0.8750 - val_loss: 0.3252 - val_acc: 0.8580
Epoch 12/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.2844 - acc: 0.8831 - val_loss: 0.3188 - val_acc: 0.8610
Epoch 13/30
9000/9000 [==============================] - 93s 10ms/step - loss: 0.2709 - acc: 0.8914 - val_loss: 0.3232 - val_acc: 0.8650
Epoch 14/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.2516 - acc: 0.8993 - val_loss: 0.3348 - val_acc: 0.8690
Epoch 15/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.2310 - acc: 0.9057 - val_loss: 0.3805 - val_acc: 0.8660
Epoch 16/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.2165 - acc: 0.9127 - val_loss: 0.3920 - val_acc: 0.8600
Epoch 17/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.1978 - acc: 0.9218 - val_loss: 0.3887 - val_acc: 0.8630
Epoch 18/30
9000/9000 [==============================] - 92s 10ms/step - loss: 0.1922 - acc: 0.9230 - val_loss: 0.4065 - val_acc: 0.8660
Epoch 19/30
9000/9000 [==============================] - 31556s 4s/step - loss: 0.1834 - acc: 0.9252 - val_loss: 0.3735 - val_acc: 0.8760
Epoch 20/30
9000/9000 [==============================] - 92s 10ms/step - loss: 0.1713 - acc: 0.9322 - val_loss: 0.3961 - val_acc: 0.8680
Epoch 21/30
9000/9000 [==============================] - 92s 10ms/step - loss: 0.1560 - acc: 0.9413 - val_loss: 0.3914 - val_acc: 0.8730
Epoch 22/30
9000/9000 [==============================] - 89s 10ms/step - loss: 0.1380 - acc: 0.9487 - val_loss: 0.4204 - val_acc: 0.8640
Epoch 23/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.1375 - acc: 0.9454 - val_loss: 0.4249 - val_acc: 0.8720
Epoch 24/30
9000/9000 [==============================] - 90s 10ms/step - loss: 0.1258 - acc: 0.9532 - val_loss: 0.4790 - val_acc: 0.8640
Epoch 25/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.1163 - acc: 0.9578 - val_loss: 0.5106 - val_acc: 0.8640
Epoch 26/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.1164 - acc: 0.9571 - val_loss: 0.5215 - val_acc: 0.8690
Epoch 27/30
9000/9000 [==============================] - 91s 10ms/step - loss: 0.1012 - acc: 0.9618 - val_loss: 0.4882 - val_acc: 0.8700
Epoch 28/30
9000/9000 [==============================] - 92s 10ms/step - loss: 0.0959 - acc: 0.9618 - val_loss: 0.5731 - val_acc: 0.8610
Epoch 29/30
9000/9000 [==============================] - 93s 10ms/step - loss: 0.0907 - acc: 0.9652 - val_loss: 0.4812 - val_acc: 0.8750
Epoch 30/30
9000/9000 [==============================] - 92s 10ms/step - loss: 0.0861 - acc: 0.9690 - val_loss: 0.5491 - val_acc: 0.8770
1000/1000 [==============================] - 4s 4ms/step
[0.5491372299194336, 0.877]
1000/1000 [==============================] - 4s 4ms/step
RNN-GRU, Word Embeddings [0.5491372299194336, 0.877]

Layer (type)                 Output Shape              Param #   
=================================================================
input_12 (InputLayer)        (None, 70)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 70, 300)           0         
_________________________________________________________________
gru_4 (GRU)                  (None, 300)               540900    
_________________________________________________________________
dropout_21 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                1010      
_________________________________________________________________
dropout_23 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 11        
=================================================================
Total params: 10,613,021
Trainable params: 572,021
Non-trainable params: 10,041,000
_________________________________________________________________

In [26]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 70)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 70, 300)           0         
_________________________________________________________________
gru_4 (GRU)                  (None, 300)               540900    
_________________________________________________________________
dropout_21 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
__________

In [ ]:
#3.7.4 Bidirectional RNN
def create_bidirectional_rnn():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.5)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(300))(embedding_layer)
    lstm_layer = layers.Dropout(0.5)(lstm_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.5)(output_layer1)
    output_layer11 = layers.Dense(10, activation="relu")(output_layer1)
    output_layer11 = layers.Dropout(0.5)(output_layer11)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer11)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',metrics=['acc'])
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model_new(classifier, train_seq_x, train_y, valid_seq_x, 30, is_neural_net=True)
print ("RNN-Bidirectional, Word Embeddings",  accuracy)

Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 167s 19ms/step - loss: 0.6843 - acc: 0.5420 - val_loss: 0.6654 - val_acc: 0.6170
Epoch 2/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.6287 - acc: 0.6533 - val_loss: 0.4747 - val_acc: 0.7770
Epoch 3/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.4994 - acc: 0.7771 - val_loss: 0.3732 - val_acc: 0.8500
Epoch 4/30
9000/9000 [==============================] - 192s 21ms/step - loss: 0.4586 - acc: 0.8027 - val_loss: 0.3502 - val_acc: 0.8590
Epoch 5/30
9000/9000 [==============================] - 188s 21ms/step - loss: 0.4276 - acc: 0.8099 - val_loss: 0.3475 - val_acc: 0.8470
Epoch 6/30
9000/9000 [==============================] - 168s 19ms/step - loss: 0.4209 - acc: 0.8263 - val_loss: 0.3386 - val_acc: 0.8500
Epoch 7/30
9000/9000 [==============================] - 168s 19ms/step - loss: 0.3984 - acc: 0.8281 - val_loss: 0.3681 - val_acc: 

In [ ]:
Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 170s 19ms/step - loss: 0.6566 - acc: 0.6051 - val_loss: 0.4907 - val_acc: 0.7900
Epoch 2/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.4919 - acc: 0.7726 - val_loss: 0.3375 - val_acc: 0.8590
Epoch 3/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.3972 - acc: 0.8248 - val_loss: 0.3279 - val_acc: 0.8530
Epoch 4/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.3786 - acc: 0.8420 - val_loss: 0.3419 - val_acc: 0.8580
Epoch 5/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.3536 - acc: 0.8511 - val_loss: 0.3066 - val_acc: 0.8700
Epoch 6/30
9000/9000 [==============================] - 165s 18ms/step - loss: 0.3300 - acc: 0.8623 - val_loss: 0.3159 - val_acc: 0.8690
Epoch 7/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.3121 - acc: 0.8710 - val_loss: 0.3129 - val_acc: 0.8680
Epoch 8/30
9000/9000 [==============================] - 167s 19ms/step - loss: 0.2770 - acc: 0.8861 - val_loss: 0.3069 - val_acc: 0.8630
Epoch 9/30
9000/9000 [==============================] - 182s 20ms/step - loss: 0.2652 - acc: 0.8920 - val_loss: 0.3224 - val_acc: 0.8730
Epoch 10/30
9000/9000 [==============================] - 171s 19ms/step - loss: 0.2354 - acc: 0.9099 - val_loss: 0.3441 - val_acc: 0.8720
Epoch 11/30
9000/9000 [==============================] - 169s 19ms/step - loss: 0.2161 - acc: 0.9143 - val_loss: 0.3517 - val_acc: 0.8550
Epoch 12/30
9000/9000 [==============================] - 170s 19ms/step - loss: 0.1884 - acc: 0.9272 - val_loss: 0.3586 - val_acc: 0.8740
Epoch 13/30
9000/9000 [==============================] - 172s 19ms/step - loss: 0.1819 - acc: 0.9309 - val_loss: 0.3743 - val_acc: 0.8730
Epoch 14/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.1529 - acc: 0.9437 - val_loss: 0.4130 - val_acc: 0.8610
Epoch 15/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.1384 - acc: 0.9473 - val_loss: 0.3638 - val_acc: 0.8760
Epoch 16/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.1238 - acc: 0.9557 - val_loss: 0.4668 - val_acc: 0.8570
Epoch 17/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.1054 - acc: 0.9626 - val_loss: 0.4543 - val_acc: 0.8620
Epoch 18/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.1016 - acc: 0.9640 - val_loss: 0.4777 - val_acc: 0.8620
Epoch 19/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.0858 - acc: 0.9689 - val_loss: 0.5529 - val_acc: 0.8680
Epoch 20/30
9000/9000 [==============================] - 165s 18ms/step - loss: 0.0776 - acc: 0.9749 - val_loss: 0.5531 - val_acc: 0.8670
Epoch 21/30
9000/9000 [==============================] - 165s 18ms/step - loss: 0.0709 - acc: 0.9760 - val_loss: 0.5856 - val_acc: 0.8690
Epoch 22/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.0729 - acc: 0.9766 - val_loss: 0.5536 - val_acc: 0.8570
Epoch 23/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.0672 - acc: 0.9782 - val_loss: 0.6229 - val_acc: 0.8730
Epoch 24/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.0705 - acc: 0.9773 - val_loss: 0.5098 - val_acc: 0.8600
Epoch 25/30
9000/9000 [==============================] - 166s 18ms/step - loss: 0.0589 - acc: 0.9821 - val_loss: 0.6825 - val_acc: 0.8570
Epoch 26/30
9000/9000 [==============================] - 168s 19ms/step - loss: 0.0481 - acc: 0.9854 - val_loss: 0.5656 - val_acc: 0.8550
Epoch 27/30
9000/9000 [==============================] - 183s 20ms/step - loss: 0.0487 - acc: 0.9829 - val_loss: 0.6314 - val_acc: 0.8600
Epoch 28/30
9000/9000 [==============================] - 162s 18ms/step - loss: 0.0440 - acc: 0.9850 - val_loss: 0.7353 - val_acc: 0.8430
Epoch 29/30
9000/9000 [==============================] - 162s 18ms/step - loss: 0.0403 - acc: 0.9880 - val_loss: 0.6665 - val_acc: 0.8580
Epoch 30/30
9000/9000 [==============================] - 162s 18ms/step - loss: 0.0474 - acc: 0.9859 - val_loss: 0.6731 - val_acc: 0.8520
1000/1000 [==============================] - 6s 6ms/step
[0.6730928390026093, 0.852]
1000/1000 [==============================] - 6s 6ms/step
RNN-Bidirectional, Word Embeddings [0.6730928390026093, 0.852]

Layer (type)                 Output Shape              Param #   
=================================================================
input_13 (InputLayer)        (None, 70)                0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 70, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 600)               1081800   
_________________________________________________________________
dropout_24 (Dropout)         (None, 600)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 100)               60100     
_________________________________________________________________
dropout_25 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 10)                1010      
_________________________________________________________________
dropout_26 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 11        
=================================================================
Total params: 11,183,921
Trainable params: 1,142,921
Non-trainable params: 10,041,000


In [28]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 70)                0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 70, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 600)               1081800   
_________________________________________________________________
dropout_24 (Dropout)         (None, 600)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 100)               60100     
_________________________________________________________________
dropout_25 (Dropout)         (None, 100)               0         
__________

In [ ]:
#3.7.5 Recurrent Convolutional Neural Network
def create_rcnn():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.5)(embedding_layer)
       
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(500, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    #pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(300))(conv_layer)
    rnn_layer = layers.Dropout(0.5)(rnn_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(20, activation="relu")(rnn_layer)
    output_layer1 = layers.Dropout(0.5)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy',metrics=['acc'])
    
    return model

classifier = create_rcnn()
accuracy = train_model_new(classifier, train_seq_x, train_y, valid_seq_x,30, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 301s 33ms/step - loss: 0.6617 - acc: 0.5894 - val_loss: 0.4991 - val_acc: 0.7950
Epoch 2/30
9000/9000 [==============================] - 301s 33ms/step - loss: 0.4963 - acc: 0.7699 - val_loss: 0.3470 - val_acc: 0.8660
Epoch 3/30
9000/9000 [==============================] - 298s 33ms/step - loss: 0.4266 - acc: 0.8123 - val_loss: 0.3758 - val_acc: 0.8340
Epoch 4/30
9000/9000 [==============================] - 296s 33ms/step - loss: 0.3926 - acc: 0.8306 - val_loss: 0.3366 - val_acc: 0.8560
Epoch 5/30
9000/9000 [==============================] - 296s 33ms/step - loss: 0.3621 - acc: 0.8486 - val_loss: 0.3205 - val_acc: 0.8800
Epoch 6/30
9000/9000 [==============================] - 297s 33ms/step - loss: 0.3321 - acc: 0.8627 - val_loss: 0.3245 - val_acc: 0.8630
Epoch 7/30
9000/9000 [=========================

In [ ]:
Train on 9000 samples, validate on 1000 samples
Epoch 1/30
9000/9000 [==============================] - 508s 56ms/step - loss: 0.6698 - acc: 0.5760 - val_loss: 0.5965 - val_acc: 0.7030
Epoch 2/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.4760 - acc: 0.7843 - val_loss: 0.3648 - val_acc: 0.8420
Epoch 3/30
9000/9000 [==============================] - 500s 56ms/step - loss: 0.3904 - acc: 0.8278 - val_loss: 0.3330 - val_acc: 0.8490
Epoch 4/30
9000/9000 [==============================] - 499s 55ms/step - loss: 0.3552 - acc: 0.8468 - val_loss: 0.3234 - val_acc: 0.8660
Epoch 5/30
9000/9000 [==============================] - 500s 56ms/step - loss: 0.3202 - acc: 0.8631 - val_loss: 0.3213 - val_acc: 0.8630
Epoch 6/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.2684 - acc: 0.8912 - val_loss: 0.3257 - val_acc: 0.8700
Epoch 7/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.2255 - acc: 0.9134 - val_loss: 0.3211 - val_acc: 0.8700
Epoch 8/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.1791 - acc: 0.9317 - val_loss: 0.3429 - val_acc: 0.8670
Epoch 9/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.1513 - acc: 0.9439 - val_loss: 0.3996 - val_acc: 0.8600
Epoch 10/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.1298 - acc: 0.9538 - val_loss: 0.3953 - val_acc: 0.8660
Epoch 11/30
9000/9000 [==============================] - 521s 58ms/step - loss: 0.1024 - acc: 0.9608 - val_loss: 0.4531 - val_acc: 0.8720
Epoch 12/30
9000/9000 [==============================] - 516s 57ms/step - loss: 0.0894 - acc: 0.9660 - val_loss: 0.4921 - val_acc: 0.8700
Epoch 13/30
9000/9000 [==============================] - 508s 56ms/step - loss: 0.0799 - acc: 0.9708 - val_loss: 0.5470 - val_acc: 0.8540
Epoch 14/30
9000/9000 [==============================] - 507s 56ms/step - loss: 0.0759 - acc: 0.9737 - val_loss: 0.4952 - val_acc: 0.8800
Epoch 15/30
9000/9000 [==============================] - 506s 56ms/step - loss: 0.0622 - acc: 0.9796 - val_loss: 0.6530 - val_acc: 0.8510
Epoch 16/30
9000/9000 [==============================] - 507s 56ms/step - loss: 0.0618 - acc: 0.9801 - val_loss: 0.5974 - val_acc: 0.8580
Epoch 17/30
9000/9000 [==============================] - 534s 59ms/step - loss: 0.0593 - acc: 0.9786 - val_loss: 0.5809 - val_acc: 0.8770
Epoch 18/30
9000/9000 [==============================] - 548s 61ms/step - loss: 0.0467 - acc: 0.9843 - val_loss: 0.5605 - val_acc: 0.8740
Epoch 19/30
9000/9000 [==============================] - 514s 57ms/step - loss: 0.0431 - acc: 0.9852 - val_loss: 0.6432 - val_acc: 0.8720
Epoch 20/30
9000/9000 [==============================] - 504s 56ms/step - loss: 0.0493 - acc: 0.9833 - val_loss: 0.6313 - val_acc: 0.8680
Epoch 21/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.0368 - acc: 0.9867 - val_loss: 0.6529 - val_acc: 0.8680
Epoch 22/30
9000/9000 [==============================] - 508s 56ms/step - loss: 0.0408 - acc: 0.9872 - val_loss: 0.7504 - val_acc: 0.8790
Epoch 23/30
9000/9000 [==============================] - 502s 56ms/step - loss: 0.0394 - acc: 0.9853 - val_loss: 0.6455 - val_acc: 0.8670
Epoch 24/30
9000/9000 [==============================] - 501s 56ms/step - loss: 0.0458 - acc: 0.9838 - val_loss: 0.6057 - val_acc: 0.8740
Epoch 25/30
9000/9000 [==============================] - 514s 57ms/step - loss: 0.0376 - acc: 0.9877 - val_loss: 0.7650 - val_acc: 0.8580
Epoch 26/30
9000/9000 [==============================] - 525s 58ms/step - loss: 0.0341 - acc: 0.9889 - val_loss: 0.7672 - val_acc: 0.8600
Epoch 27/30
9000/9000 [==============================] - 514s 57ms/step - loss: 0.0314 - acc: 0.9892 - val_loss: 0.7108 - val_acc: 0.8590
Epoch 28/30
9000/9000 [==============================] - 517s 57ms/step - loss: 0.0376 - acc: 0.9884 - val_loss: 0.6822 - val_acc: 0.8700
Epoch 29/30
9000/9000 [==============================] - 516s 57ms/step - loss: 0.0326 - acc: 0.9890 - val_loss: 0.6904 - val_acc: 0.8770
Epoch 30/30
9000/9000 [==============================] - 516s 57ms/step - loss: 0.0312 - acc: 0.9893 - val_loss: 0.7660 - val_acc: 0.8640
1000/1000 [==============================] - 17s 17ms/step
[0.7660071992874146, 0.864]
1000/1000 [==============================] - 17s 17ms/step
CNN, Word Embeddings [0.7660071992874146, 0.864]

Layer (type)                 Output Shape              Param #   
=================================================================
input_18 (InputLayer)        (None, 70)                0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_19 (Spatia (None, 70, 300)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 68, 1000)          901000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 600)               2341800   
_________________________________________________________________
dropout_36 (Dropout)         (None, 600)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 20)                12020     
_________________________________________________________________
dropout_37 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 21        
=================================================================
Total params: 13,295,841
Trainable params: 3,254,841
Non-trainable params: 10,041,000
_________________________________________________________________



In [34]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 70)                0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 70, 300)           10041000  
_________________________________________________________________
spatial_dropout1d_19 (Spatia (None, 70, 300)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 68, 1000)          901000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 600)               2341800   
_________________________________________________________________
dropout_36 (Dropout)         (None, 600)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 20)                12020     
__________